Summarization with legal-pegasus

In [ ]:
import functions as fct
import bm25s
import numpy as np
import json
import time
import pandas as pd

In [ ]:
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'
train_path_base = 'data_txt_save/train_'
clean_path_base = 'clean_data_txt_save/train_'

document = fct.open_file(train_path, "json")

# Boucle sur les 10 premiers textes
for i in range(10):
    # Construire les chemins pour chaque fichier en fonction de l'index
    train_path_txt = f'{train_path_base}{i}.txt'
    clean_path_txt = f'{clean_path_base}{i}.txt'
    train = document[i]["raw_source"]

    # Ouvre les fichiers correspondants
    document_txt = fct.open_file(train_path_txt, "txt")
    clean_document_txt = fct.open_file(clean_path_txt, "txt")
    
    # Résumer les fichiers
    summary_txt = fct.summarize(document_txt, "legal-pegasus", max_length=1000)
    summary_clean_txt = fct.summarize(clean_document_txt, "legal-pegasus", max_length=1000)
    summary_raw = fct.summarize(train, "legal-pegasus", max_length=1000)

    # Afficher les résumés
    print(f"Summary TXT {i+1}:", summary_txt)
    print(f"Summary Clean TXT {i+1}:", summary_clean_txt)
    print(f"Summary Raw {i+1}:", summary_raw)

In [ ]:
import pandas as pd

# Chemins des fichiers
train_path = 'SCOTUS/train.json'
train_path_base = 'data_txt_save/train_'
clean_path_base = 'clean_data_txt_save/train_'

# Ouvre le fichier JSON
document = fct.open_file(train_path, "json")

# Stocker les données sous forme de DataFrame
results = []
execution_times = {}

# Boucle sur les 10 premiers textes
for i in range(10):
    # Construire les chemins pour chaque fichier
    train_path_txt = f'{train_path_base}{i}.txt'
    clean_path_txt = f'{clean_path_base}{i}.txt'
    
    # Ouvrir les fichiers correspondants
    document_txt = fct.open_file(train_path_txt, "txt")
    clean_document_txt = fct.open_file(clean_path_txt, "txt")
    train = document[i]["raw_source"]
    
    # Résumer les fichiers
    summary_txt = fct.summarize(document_txt, "legal-pegasus", max_length=1000)
    summary_clean_txt = fct.summarize(clean_document_txt, "legal-pegasus", max_length=1000)
    summary_raw = fct.summarize(train, "legal-pegasus", max_length=1000)

    # Calculer les longueurs des résumés comme une métrique simple
    len_summary_txt = len(summary_txt)
    len_summary_clean_txt = len(summary_clean_txt)
    len_summary_raw = len(summary_raw)

    # Ajouter les résultats à la liste
    results.append({
        'Text ID': i,
        'Method': 'TXT',
        'Summary Length': len_summary_txt,
        'Summary': summary_txt
    })
    results.append({
        'Text ID': i,
        'Method': 'Clean TXT',
        'Summary Length': len_summary_clean_txt,
        'Summary': summary_clean_txt
    })
    results.append({
        'Text ID': i,
        'Method': 'Raw',
        'Summary Length': len_summary_raw,
        'Summary': summary_raw
    })

    break

# Convertir les résultats en DataFrame
df = pd.DataFrame(results)

# Réarranger les colonnes
df = df[['Text ID', 'Method', 'Summary Length', 'Summary']]

# Fonction de surbrillance pour les longueurs de résumés
def highlight_length(df):
    styles = pd.DataFrame('', index=df.index, columns=df.columns)

    # Appliquer la surbrillance pour les 3 plus longues et 3 plus courtes longueurs de résumés
    top_3_max = df['Summary Length'].nlargest(3)
    top_3_min = df['Summary Length'].nsmallest(3)

    # Dégradé vert pour les plus longs résumés
    for i in df.index:
        if df['Summary Length'].iloc[i] in top_3_max.values:
            rank = top_3_max.rank(ascending=False)[top_3_max == df['Summary Length'].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3
            styles.loc[i, 'Summary Length'] = f'background-color: rgba(50, 200, 50, {alpha});'

    # Dégradé rouge pour les plus courts résumés
    for i in df.index:
        if df['Summary Length'].iloc[i] in top_3_min.values:
            rank = top_3_min.rank()[top_3_min == df['Summary Length'].iloc[i]].values[0]
            alpha = 1 - (rank - 1) / 3  
            styles.loc[i, 'Summary Length'] = f'background-color: rgba(200, 50, 50, {alpha});'

    return styles

# Appliquer le style au DataFrame
styled_df = df.style.apply(highlight_length, axis=None)

# Afficher le DataFrame stylisé
styled_df


In [ ]:
import pandas as pd
import functions as fct
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch
import os


train_path = 'dataset_legal-pegasus/dataset/UK-Abs/train-data'
test_path = 'dataset_legal-pegasus/dataset/UK-Abs/test-data'

train_path_txt = train_path + '/judgement'
train_path_summary = train_path + '/summary'
test_path_txt = test_path + '/judgement'
test_path_summary = test_path + '/summary'

# Parcourir les fichiers dans le dossier de train et de test
train_files = os.listdir(train_path_txt)
test_files = os.listdir(test_path_txt)

# Charger le modèle Legal-Pegasus et le tokenizer
model_name = "nsi319/legal-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

In [2]:
# Fonction pour diviser le document en segments
def chunk_text(text, chunk_size=1024):
    inputs = tokenizer(text, return_tensors="pt", max_length=chunk_size, truncation=True, padding=True)
    chunks = []
    current_chunk = []
    current_length = 0
    
    for i, word in enumerate(text.split()):
        if current_length + len(word) <= chunk_size:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1

    if current_chunk:
        chunks.append(" ".join(current_chunk))
        
    return chunks

In [ ]:
reference_summary = fct.open_file(train_path_txt + '/' + train_files[0], 'txt')
print(len(reference_summary))

# Générer le résumé
chunks = chunk_text(reference_summary)
print(len(chunks))

# generated_summary = [fct.summarize(chunk, "legal-pegasus", max_length=1000) for chunk in chunks]
generated_summary = fct.summarize(chunks[0], "legal-pegasus", max_length=1000)

In [ ]:
print("Résumé de référence : ", reference_summary)
print("Résumé généré : ", generated_summary)

In [ ]:
from rouge import Rouge

# Fonction pour calculer les scores ROUGE
def compute_rouge_scores(reference_summary, generated_summary):
    rouge = Rouge()
    
    # Calcul des scores ROUGE-1, ROUGE-2 et ROUGE-L
    scores = rouge.get_scores(generated_summary, reference_summary)
    
    # S'assurer que des scores ont été calculés avant de procéder
    if len(scores) == 0:
        return {"error": "No scores could be computed. Check your input summaries."}
    
    # Calcul de la moyenne des scores
    avg_scores = {
        'rouge-1': {
            'precision': sum([score['rouge-1']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-1']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-1']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        },
        'rouge-2': {
            'precision': sum([score['rouge-2']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-2']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-2']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        },
        'rouge-l': {
            'precision': sum([score['rouge-l']['p'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'recall': sum([score['rouge-l']['r'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
            'f1-score': sum([score['rouge-l']['f'] for score in scores]) / len(scores) if len(scores) > 0 else 0,
        }
    }
    
    return avg_scores

# Calcul des scores ROUGE
rouge_scores = compute_rouge_scores(reference_summary, generated_summary)

# Vérification et affichage des scores
if "error" in rouge_scores:
    print(rouge_scores["error"])
else:
    print("ROUGE-1: ", rouge_scores['rouge-1'])
    print("ROUGE-2: ", rouge_scores['rouge-2'])
    print("ROUGE-L: ", rouge_scores['rouge-l'])


In [ ]:
from bert_score import score

# Calculer le score BERTScore
P, R, F1 = score(generated_summary, reference_summary, lang="en", verbose=True)

# Afficher les scores de Précision, Rappel et F1
print(f"Précision (P): {P.mean().item():.4f}")
print(f"Rappel (R): {R.mean().item():.4f}")
print(f"F1-Score: {F1.mean().item():.4f}")
